# Campaign Bried Creation

In this campaign brief creation, 
We have 3 Agents
1. Supervisor Agent: Manages the entire agents workflow
2. Brief Creator Agent: Create the brief according the given prompt
3. Summariser Agent: Summarises the entire brief from the previously existing data.

and we have 3 Tools:
1. extract_placeholders_from_template: Extracts the placeholders from the template
2. aggregate_text_files: Stich all the previous campaign brief txt files as a single file and pass it with LLM's.
3. populate_word_from_json: Write back the content into the word file


In [ ]:
#Basic import for LLM and Langchain Agents
import os
from langchain.agents import Tool, initialize_agent
from langchain_community.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain_openai import AzureChatOpenAI

In [ ]:
#Initializing the Azure OpenAI 4o Mini LLM
llm = AzureChatOpenAI(model="gpt-4o-mini",
    api_version="2025-01-01-preview",
    azure_endpoint="",
    api_key=""
)

In [ ]:
#Tool 1: Extracts the placeholders from the templates

# ----- START: Code Block for Defining extract_placeholders_tool (Refinement Removed) -----

import re # Import regular expressions module
import os
from typing import List, Dict, Any # Import necessary types
# from langchain_core.pydantic_v1 import BaseModel, Field # Using v1 for consistency
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool
import docx # Requires: pip install python-docx

print("--- Defining Word placeholder extraction tool dependencies and function (Refinement Removed) ---")

# --- Pydantic Schema for Tool Arguments ---
class ExtractPlaceholdersArgs(BaseModel):
    """Input schema for the ExtractPlaceholdersTool."""
    template_path: str = Field(description="Path to the Word template (.docx) file containing placeholders like {{PLACEHOLDER_NAME}}.")

# --- Placeholder Refinement Mapping (REMOVED) ---
# placeholder_refinement_map: Dict[str, str] = {
#     "{{PLACEHOLDER_ROLES}}": "{{PLACEHOLDER_ROLES_AND_RESPONSIBILITIES}}",
# }

# --- Core Python Function (Refinement Removed) ---
def extract_placeholders_func(template_path: str) -> Dict[str, Any]:
    """
    (Refinement Removed) Extracts placeholders (like {{PLACEHOLDER_NAME}})
    from a Word document template and returns the list of unique placeholders exactly as found.

    Args:
        template_path: Path to the .docx template file.

    Returns:
        A dictionary containing:
         - 'extracted_placeholders': A list of unique placeholder strings found (exactly as in template).
         - 'status': A success or error message string.
    """
    print(f"\n--- Running extract_placeholders_func (Refinement Removed) ---")
    print(f"Attempting to extract placeholders from template '{template_path}'...")
    absolute_template_path = os.path.abspath(template_path)
    print(f"DEBUG: Absolute Template Path: {absolute_template_path}")

    if not os.path.exists(template_path):
        error_msg = f"Error: Template file not found at '{template_path}'"
        print(error_msg)
        return {"extracted_placeholders": [], "status": error_msg}

    try:
        doc = docx.Document(template_path)
        placeholder_regex = re.compile(r"\{\{\s*(.*?)\s*\}\}")
        found_placeholders = set()

        # --- Function to search within text runs (Refinement Removed) ---
        def find_in_runs(runs):
             full_text = "".join(run.text for run in runs)
             matches = placeholder_regex.findall(full_text)
             for match_content in matches:
                 # CHANGE: Use original placeholder directly, no refinement lookup
                 original_placeholder = f"{{{{{match_content}}}}}"
                 found_placeholders.add(original_placeholder)
                 # CHANGE: Simplified print statement
                 print(f"  Found '{original_placeholder}'")

        # --- Search in regular paragraphs ---
        print("\nChecking regular paragraphs...")
        for paragraph in doc.paragraphs:
            if '{{' in paragraph.text and '}}' in paragraph.text:
                 find_in_runs(paragraph.runs)

        # --- Search within tables ---
        print("\nChecking tables...")
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                         if '{{' in paragraph.text and '}}' in paragraph.text:
                             find_in_runs(paragraph.runs)

        if not found_placeholders:
            status_msg = f"No placeholders like {{...}} found in '{template_path}'."
            print(status_msg)
            return {"extracted_placeholders": [], "status": status_msg}
        else:
            sorted_placeholders = sorted(list(found_placeholders))
            status_msg = f"Successfully extracted {len(sorted_placeholders)} unique placeholders from '{template_path}'."
            print(status_msg)
            print(f"Placeholders found: {sorted_placeholders}")
            return {"extracted_placeholders": sorted_placeholders, "status": status_msg}

    except Exception as e:
        error_msg = f"Error during placeholder extraction from '{template_path}': {e}"
        print(error_msg)
        return {"extracted_placeholders": [], "status": error_msg}

# --- Create the LangChain StructuredTool (Updated Description) ---
extract_placeholders_tool = StructuredTool.from_function(
    func=extract_placeholders_func,
    name="extract_placeholders_from_template",
    # CHANGE: Update description to remove mention of refinement
    description="Reads a Word document (.docx) template, extracts all unique placeholders like {{PLACEHOLDER_NAME}} found within it (in paragraphs and tables), and returns them as a list exactly as they appear in the template.",
    args_schema=ExtractPlaceholdersArgs,
    return_direct=False
)

print(f"--- Successfully defined tool: {extract_placeholders_tool.name} (Refinement Removed) ---")

# ----- END: Code Block for Defining extract_placeholders_tool (Refinement Removed) -----

In [ ]:
#Tool 2: Writing down the newly generated output from agent into word document

# ----- START: Modified Code Block for populate_word_from_json_func (More Robust Key Handling) -----

import os
import json
from typing import Dict, Any
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool
import docx

print("--- Defining Word population tool dependencies and function (Robust Key Handling) ---")

# --- Pydantic Schema for Tool Arguments ---
class PopulateWordArgs(BaseModel):
    """Input schema for the PopulateWordFromJSONTool."""
    # CHANGE: Reflect more flexible key handling
    json_data: Dict[str, Any] = Field(description="JSON data (as a Python dictionary). Keys should ideally match the content inside placeholders (e.g., 'PLACEHOLDER_CAMPAIGN_NAME' or just 'CAMPAIGN_NAME'), NOT the full '{{...}}'.")
    template_path: str = Field(description="Path to the Word template (.docx) file containing placeholders like {{PLACEHOLDER_KEY}}.")
    output_path: str = Field(description="Path where the populated Word document will be saved.")

# --- Core Python Function (Modified for Robust Key Handling) ---
def populate_word_from_json_func(json_data: Dict[str, Any], template_path: str, output_path: str) -> str:
    """
    (Robust Keys) Populates a Word document template with data from a JSON object.
    Placeholders in the Word document should be like {{PLACEHOLDER_KEY}}.
    The keys in the input json_data dictionary can be either 'PLACEHOLDER_KEY' or 'KEY'.
    The function will construct the correct '{{PLACEHOLDER_KEY}}' to search for.
    Includes debugging.

    Args:
        json_data: Dictionary containing the data. Keys expected without braces.
        template_path: Path to the .docx template file.
        output_path: Path to save the populated .docx file.

    Returns:
        A success or error message string.
    """
    print(f"\n--- Running populate_word_from_json_func (Robust Keys) ---")
    print(f"Attempting to populate template '{template_path}'...")
    if not isinstance(json_data, dict):
        msg = f"ERROR: Input 'json_data' is not a dictionary (received type: {type(json_data)}). Cannot proceed."
        print(msg)
        return msg

    print(f"DEBUG: Received json_data keys: {list(json_data.keys())}")

    # --- Path checking remains the same ---
    try:
        cwd = os.getcwd()
        absolute_template_path = os.path.abspath(template_path)
        absolute_output_path = os.path.abspath(output_path)
    except Exception as path_e:
        print(f"DEBUG: Error getting path info: {path_e}")
        absolute_template_path = template_path # Fallback
        absolute_output_path = output_path # Fallback

    print(f"DEBUG: Absolute Template Path: {absolute_template_path}")
    print(f"DEBUG: Absolute Output Path: {absolute_output_path}")


    try:
        if not os.path.exists(template_path):
             print(f"ERROR: Template file not found at relative path: '{template_path}'")
             if not os.path.exists(absolute_template_path):
                 print(f"ERROR: Template file also not found at absolute path: '{absolute_template_path}'")
             return f"Error: Template file not found at '{template_path}'"

        doc = docx.Document(template_path)
        placeholders_replaced_count = 0
        # Use the keys received from the JSON dict
        keys_from_json = list(json_data.keys())
        keys_successfully_replaced = set() # Track keys that were actually replaced

        # --- Helper Function (Modified for Robust Key Handling) ---
        def find_and_replace_in_paragraph(paragraph, current_json_keys):
            nonlocal placeholders_replaced_count, keys_successfully_replaced
            text_replaced_in_para = False

            # Iterate through the keys provided in the JSON data
            for key_from_json in current_json_keys:
                # CHANGE: Construct the full placeholder to search for, adding 'PLACEHOLDER_' if missing
                content_key = key_from_json # Start with the key as received
                if not content_key.upper().startswith("PLACEHOLDER_"):
                    placeholder_to_find = f"{{{{PLACEHOLDER_{content_key}}}}}"
                else:
                    placeholder_to_find = f"{{{{{content_key}}}}}"

                # Search for the constructed placeholder in the paragraph text
                if placeholder_to_find in paragraph.text:
                    text_to_insert = str(json_data[key_from_json])
                    # Replace using runs for robustness
                    inline = paragraph.runs
                    for i in range(len(inline)):
                        if placeholder_to_find in inline[i].text:
                            text = inline[i].text.replace(placeholder_to_find, text_to_insert)
                            inline[i].text = text
                            print(f"  Replaced '{placeholder_to_find}' using key '{key_from_json}' in paragraph starting: '{paragraph.text[:50]}...'")
                            keys_successfully_replaced.add(key_from_json) # Track the key from JSON that was used
                            text_replaced_in_para = True
                            # Assuming one replacement per key per paragraph pass for simplicity
                            # If multiple instances of the same placeholder exist, this might need adjustment

            return text_replaced_in_para

        # --- Search logic remains largely the same ---
        print("\nChecking regular paragraphs...")
        remaining_keys = list(keys_from_json) # Start with all keys
        find_and_replace_in_paragraph(doc.paragraphs[0], remaining_keys) # Example: Process first paragraph

        for paragraph in doc.paragraphs:
           find_and_replace_in_paragraph(paragraph, remaining_keys)


        print("\nChecking tables...")
        if remaining_keys: # Only check tables if keys might still need replacing
            for table_idx, table in enumerate(doc.tables):
                for row_idx, row in enumerate(table.rows):
                    for cell_idx, cell in enumerate(row.cells):
                        for para_idx, paragraph in enumerate(cell.paragraphs):
                             find_and_replace_in_paragraph(paragraph, remaining_keys)

        # Calculate counts based on keys successfully replaced
        placeholders_replaced_count = len(keys_successfully_replaced)
        keys_not_found_in_template = [k for k in keys_from_json if k not in keys_successfully_replaced]

        if keys_not_found_in_template:
            print(f"\nWARNING: Values for these JSON keys were provided, but corresponding '{{{{PLACEHOLDER_...}}}}' placeholders were NOT found in the template: {keys_not_found_in_template}")
        else:
            print("\nAll provided JSON keys corresponded to placeholders found and replaced.")
        print(f"Total unique placeholders found and replaced: {placeholders_replaced_count} out of {len(keys_from_json)} provided JSON keys.")

        # --- Directory creation and saving logic remains the same ---
        output_dir = os.path.dirname(output_path)
        if output_dir and not os.path.exists(output_dir):
             try:
                os.makedirs(output_dir)
                print(f"Created output directory: {output_dir}")
             except OSError as dir_e:
                 print(f"ERROR creating output directory '{output_dir}': {dir_e}")
                 return f"Error creating output directory: {dir_e}"

        print(f"\nAttempting to save populated document to: '{output_path}' (Absolute: '{absolute_output_path}')")
        try:
            doc.save(output_path)
            print(f"Successfully called doc.save() for: {output_path}")
            if not os.path.exists(output_path):
                 print(f"CRITICAL WARNING: File NOT found at '{output_path}' immediately after save call!")
                 if not os.path.exists(absolute_output_path):
                     print(f"CRITICAL WARNING: File also NOT found at absolute path '{absolute_output_path}'!")
                 return f"Error: File saving failed silently for {output_path}"
            else:
                print(f"File confirmed to exist at '{output_path}' after saving.")
                return f"Successfully populated template and saved to '{output_path}'"
        except Exception as e_save:
            print(f"ERROR during doc.save() or file check: {e_save}")
            if isinstance(e_save, PermissionError):
                 print(f"Hint: Check write permissions for the directory '{os.path.dirname(absolute_output_path)}'")
                 print(f"Hint: Ensure the file '{absolute_output_path}' is not already open in another application.")
            return f"Error during file save operation: {e_save}"

    except FileNotFoundError:
         return f"Error: Template file not found at '{template_path}' during processing."
    except Exception as e:
        print(f"An error occurred during Word processing: {e}")
        return f"Error populating Word template: {e}"


# --- Create the LangChain StructuredTool (Update description) ---
populate_word_tool = StructuredTool.from_function(
    func=populate_word_from_json_func,
    name="populate_word_from_json",
    # CHANGE: Update description to reflect robust key handling
    description="Populates a Word document (.docx) template (placeholders like {{PLACEHOLDER_KEY}}) using data from a JSON object. The KEYS in the input 'json_data' dictionary should match the content inside the braces, ideally including the 'PLACEHOLDER_' prefix (e.g., 'PLACEHOLDER_KEY' or 'KEY'), but NOT the full '{{...}}'.",
    args_schema=PopulateWordArgs,
)

print(f"--- Successfully defined tool: {populate_word_tool.name} (Robust Key Handling) ---")

# ----- END: Modified Code Block for populate_word_from_json_func (Robust Key Handling) -----

In [ ]:
# # Tool 3: Aggregating the previous campaign brief txt files into one together as input for summariser agent
# import os
# # from langchain.agents import Tool # Original import (can be removed if not used elsewhere)
# from langchain.tools import StructuredTool # <--- CHANGE: Import StructuredTool

# def aggregate_text_files_tool_func() -> str: # Function definition remains the same
#     """
#     Tool to aggregate content from all .txt files in the './Data/' directory.
#     """
#     directory_path = "./Data/" # Hardcoded default directory
#     print(f"aggregate_text_files_tool_func is trying to access directory: {directory_path}") # DEBUG PRINT
#     aggregated_text = ""
#     try:
#         print(f"Listing files in directory: {directory_path}") # DEBUG PRINT
#         filenames = os.listdir(directory_path)
#         print(f"Files found: {filenames}") # DEBUG PRINT
#         for filename in filenames:
#             if filename.endswith(".txt"):
#                 filepath = os.path.join(directory_path, filename)
#                 print(f"Processing file: {filepath}") # DEBUG PRINT
#                 with open(filepath, "r", encoding="utf-8") as f:
#                     aggregated_text += f.read() + "\n\n"
#         if not aggregated_text:
#             return "No .txt files found in the directory."
#         print ("Returning Aggregated Text")
#         return aggregated_text
#     except FileNotFoundError:
#         return f"Directory '{directory_path}' not found."

# # Use StructuredTool.from_function
# aggregate_data_tool = StructuredTool.from_function( # <--- CHANGE: Use StructuredTool
#     func=aggregate_text_files_tool_func,
#     name="aggregate_text_files",
#     description="Useful for aggregating content from previous campaign data files in a directory.",
#     args_schema=None # Explicitly set args_schema to None for zero-input tool still works here
# )

# # tools_for_supervisor = [aggregate_data_tool, populate_word_tool]
# # print(f"Tools available for supervisor: {[tool.name for tool in tools_for_supervisor]}") # Optional: Confirmation print
# # ----- END CELL 1 (Modified) -----

In [2]:
!pip install chromadb

  Using cached chromadb-1.0.5-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.25.0-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached onnxruntime-1.21.1-cp313-cp313-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.32.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_in

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (chroma-hnswlib)


In [1]:
# build_vector_store.py (Azure Version)
import os
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings,AzureOpenAIEmbeddings # Handles Azure via env vars
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv() # Load environment variables from .env

# --- Configuration ---
SOURCE_DIRECTORY = "./Data/"
PERSIST_DIRECTORY = "./vector_store_db"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 150
# --- Azure environment variables are expected to be loaded ---

def create_or_update_vector_store():
    """
    Loads .txt files, splits them, creates embeddings using Azure OpenAI,
    and stores them in Chroma.
    """
    print(f"Loading documents from: {SOURCE_DIRECTORY}")
    loader = DirectoryLoader(
        SOURCE_DIRECTORY, glob="**/*.txt", loader_cls=TextLoader,
        loader_kwargs={'encoding': 'utf-8'}, show_progress=True, use_multithreading=True
    )
    documents = loader.load()
    if not documents:
        print("No .txt documents found. Exiting.")
        return
    print(f"Loaded {len(documents)} documents.")

    print("Splitting documents into chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
    )
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} text chunks.")

    print("Initializing Azure OpenAI embedding model...")
    # OpenAIEmbeddings automatically uses Azure variables if OPENAI_API_TYPE=azure
    # It reads AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME for the deployment
    embeddings = AzureOpenAIEmbeddings(
        # deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"), # Usually picked up automatically
        model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"), # Often need to specify model name same as deployment for Azure
        chunk_size=1 # Recommended for Azure embeddings
    )
    print("Embedding model initialized.")

    print(f"Creating/updating vector store at: {PERSIST_DIRECTORY}")
    vectordb = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectordb.persist()
    print("Vector store created/updated and persisted successfully.")

if __name__ == "__main__":
    # Ensure .env is loaded before anything else if running directly
    load_dotenv()
    if not os.path.exists(PERSIST_DIRECTORY):
        os.makedirs(PERSIST_DIRECTORY)
    # Ensure all required env vars are present before running
    required_vars = ['OPENAI_API_TYPE', 'OPENAI_API_VERSION', 'AZURE_OPENAI_ENDPOINT', 'OPENAI_API_KEY', 'AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME']
    if not all(os.getenv(var) for var in required_vars):
        print(f"Error: Missing one or more required Azure environment variables: {required_vars}")
    else:
        create_or_update_vector_store()

Loading documents from: ./Data/


100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 831.96it/s]

Loaded 6 documents.
Splitting documents into chunks...
Split into 22 text chunks.
Initializing Azure OpenAI embedding model...


Embedding model initialized.
Creating/updating vector store at: ./vector_store_db


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
# Displaying the tools used by the Supervisor Agent
tools_for_supervisor = [extract_placeholders_tool,aggregate_data_tool, populate_word_tool]
print(f"Tools available for supervisor: {[tool.name for tool in tools_for_supervisor]}")

In [ ]:
# Agent 1: Summariser Agent: Summarises the old campaign briefs shared by aggregate_data_tool and parse to Brief Generator Agent to create new brief.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage # Import SystemMessage

# 2. Define Summarizer Prompt - USING ChatPromptTemplate and MessagesPlaceholder
# Define the system message part separately (clearer instructions for the LLM)
summarizer_system_message = """You are a Campaign Data Summarizer Agent.
Your input is a list of messages containing the conversation history.

Your specific task is:
1.  Examine the provided message history ('messages').
2.  **Locate the ToolMessage containing the output from the 'extract_placeholders_from_template' tool.** Extract the list of required `extracted_placeholders` from its content. Note these required fields.
3.  **Locate the most recent ToolMessage containing the output from the 'aggregate_text_files' tool.** Extract the raw aggregated text content from it.
4.  **Summarize the extracted raw text content.**

**When summarizing, focus on extracting key information generally relevant for creating new campaign briefs, such as:**
    - Target audience insights
    - Past campaign objectives and strategies
    - Key learnings (successes/failures)
    - Performance metrics/results
    - Observed trends/patterns and others

**IMPORTANT:** While extracting the above, **pay special attention and prioritize information that seems directly relevant to the topics indicated by the `extracted_placeholders` list** you noted in step 2. Your goal is to provide a summary that is both generally informative AND maximally useful for filling the specific fields required by the template.

Your final output should be ONLY the concise, informative summary containing the prioritized, relevant information."""

# Create the ChatPromptTemplate using MessagesPlaceholder
summarizer_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content=summarizer_system_message),
    MessagesPlaceholder(variable_name="messages") # CRITICAL: Use MessagesPlaceholder
])

# 3. Create the Summarizer Agent - Using the new prompt structure
summarizer_agent = create_react_agent(
    model=llm,
    tools=[], # No tools for the summarizer itself
    prompt=summarizer_prompt_template, # Use the MODIFIED ChatPromptTemplate
    name="summarizer_agent"
    # create_react_agent is designed to work with MessagesPlaceholder and should
    # automatically pass the 'messages' list from the input state.
)

# 4. Example Usage and Testing (Optional for now)
# ...

if __name__ == "__main__":
    print("Summarizer Agent defined successfully with ChatPromptTemplate!") # Confirmation message
# ----- END CELL 2 (Modified Again) -----

In [ ]:
# Agent 3: Brief Generator Agent: This Agent generates the new brief acc to the input prompt by user and keeping the context of previous campaign with the help of summariser agent and generated the JSON output as a new brief.
from langchain_openai import ChatOpenAI # Updated import if using newer langchain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import SystemMessage

# 2. Define Brief Generator Prompt - ENHANCED FOR COMPLETENESS
#    Focus is on ensuring ALL placeholders from the tool output are addressed.
brief_generator_system_message = """You are a meticulous Campaign Brief Generator Agent.
Your primary goal is to create a comprehensive draft campaign brief by synthesizing information and ensuring ALL required sections are included.

Your input is a list of messages representing the conversation history.

**Your Task Breakdown:**

1.  **Identify Inputs:** Carefully examine the provided message history ('messages') to locate:
    *   The initial `HumanMessage` containing the user's original request and core requirements.
    *   The most recent `AIMessage` (likely from a 'summarizer_agent') containing relevant summarized data.
    *   The `ToolMessage` containing the output from the 'extract_placeholders_from_template' tool. **This message contains the critical list of `extracted_placeholders`.**

2.  **Confirm Placeholders:** Extract the exact list of `extracted_placeholders` from the `ToolMessage`. Let's call this the `REQUIRED_SECTIONS_LIST`.

3.  **Generate Content for ALL Placeholders:** This is the most critical step. Iterate through **every single item** in the `REQUIRED_SECTIONS_LIST`. For each placeholder string:
    *   Synthesize relevant information *specifically for that placeholder's topic* using the user's request (from step 1a) and the focused summary (from step 1b).
    *   Generate clear and concise content that directly addresses the placeholder's purpose (e.g., for `{{PLACEHOLDER_OBJECTIVES}}`, generate the campaign objectives; for `{{PLACEHOLDER_CORE_MESSAGE}}`, generate the core message).
    *   **Handling Missing Information:** If the available inputs do not contain explicit information for a specific placeholder, you MUST still include the section. Use your knowledge to infer a reasonable starting point OR clearly state 'To be determined based on [relevant factor]' or 'N/A - Requires further input'. **Crucially, DO NOT OMIT THE SECTION/PLACEHOLDER itself under any circumstances.**
4.  **Structure and Combine Output:** Assemble the generated content for all placeholders into a single, coherent campaign brief text.
    *   **Generate Content Under Headings:** Internally or in your text output, use clear headings corresponding to the placeholders to ensure you cover everything (e.g., "OBJECTIVES:", "CORE_MESSAGE:", "BUDGET:", "ASSETS:", etc.).
    *   **CRITICAL - Key Naming for Downstream Use:** Ensure that when this information is eventually structured (e.g., into JSON), the keys used **MUST EXACTLY MATCH** the placeholder names without the brackets and prefix.
    *   **Provide Explicit Mapping (Example within Prompt):**
        *   Content for `{{PLACEHOLDER_CAMPAIGN_NAME}}` must correspond to the key `CAMPAIGN_NAME`.
        *   Content for `{{PLACEHOLDER_CAMPAIGN_TYPE}}` must correspond to the key `CAMPAIGN_TYPE`.
        *   Content for `{{PLACEHOLDER_OBJECTIVES}}` must correspond to the key `OBJECTIVES`.
        *   Content for `{{PLACEHOLDER_AUDIENCE}}` must correspond to the key `AUDIENCE`.
        *   Content for `{{PLACEHOLDER_CHANNELS}}` must correspond to the key `CHANNELS`.
        *   Content for `{{PLACEHOLDER_DURATION}}` must correspond to the key `DURATION`.
        *   Content for `{{PLACEHOLDER_BUDGET}}` must correspond to the key `BUDGET`. (NOT 'BUDGET ALLOCATION')
        *   Content for `{{PLACEHOLDER_CORE_MESSAGE}}` must correspond to the key `CORE_MESSAGE`.
        *   Content for `{{PLACEHOLDER_ASSETS}}` must correspond to the key `ASSETS`. (NOT 'ASSETS REQUIRED')
        *   Content for `{{PLACEHOLDER_COMPLIANCE}}` must correspond to the key `COMPLIANCE`.
        *   Content for `{{PLACEHOLDER_TECHNICAL}}` must correspond to the key `TECHNICAL`.
        *   Content for `{{PLACEHOLDER_MEASUREMENT}}` must correspond to the key `MEASUREMENT`. (NOT 'MEASUREMENT & REPORTING')
        *   Content for `{{PLACEHOLDER_INSIGHTS}}` must correspond to the key `INSIGHTS`.
        *   Content for `{{PLACEHOLDER_ROLES}}` must correspond to the key `ROLES`. (NOT 'ROLES & RESPONSIBILITIES')
    *   **Final Check:** Before outputting, verify that you have included content for **every single placeholder** from the `REQUIRED_SECTIONS_LIST` and that the structure facilitates the correct key mapping described above.

5.  **Final Output:** Your final output MUST be ONLY the generated campaign brief text, structured clearly section by section, ready for conversion into a data structure using the precise keys listed above.
"""
# Create the ChatPromptTemplate using MessagesPlaceholder
brief_generator_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content=brief_generator_system_message),
    MessagesPlaceholder(variable_name="messages") # Input messages history
])

# 3. Create the Brief Generator Agent - Using the enhanced prompt
brief_generator_agent = create_react_agent(
    model=llm,
    tools=[], # Generator doesn't call other tools, it synthesizes
    prompt=brief_generator_prompt_template, # Use the ENHANCED ChatPromptTemplate
    name="brief_generator_agent"
    # create_react_agent maps the 'messages' list from the graph state automatically
)

# 4. Example Usage and Testing (Conceptual - depends on your LangGraph setup)
#    This agent would typically be a node in your LangGraph.
#    Input state would contain the 'messages' list including user request, summary, and placeholder tool output.
#    Output state would contain the generated brief text in the 'messages' list as an AIMessage.

if __name__ == "__main__":
    # This part is just for confirmation during development/testing setup
    print("Brief Generator Agent defined with ENHANCED prompt!")
    print("\n--- Sample Prompt Structure ---")
    # You can optionally print the template to review structure (for debugging)
    # print(brief_generator_prompt_template.pretty_print())
    print("System Message Length:", len(brief_generator_system_message)) # Check length if concerned about token limits
# ----- END CELL 3 (Modified & Enhanced Prompt) -----

In [ ]:
# 7. Define Supervisor Agent and Workflow using create_supervisor
from langgraph_supervisor import create_supervisor
supervisor_prompt = ("""You are a Campaign Brief Supervisor Agent. Your job is to manage specialized agents and tools to:
1. Identify required information fields (placeholders) from a Word template.
2. Gather background data.
3. Generate a new campaign brief based on user requirements and background data, structured according to the identified fields.
4. Populate the Word template with the generated brief content.

You have access to the following tools:

1. extract_placeholders_from_template: Reads a Word template (.docx), extracts all unique placeholders like {{PLACEHOLDER_NAME}}, and returns them as a list exactly as they appear. Requires 'template_path'. Use './Data/CampaignBriefCreationTemplate.docx'.
2. aggregate_text_files: Aggregates content from previous campaign data files in './Data/'. Does NOT require input arguments.
3. populate_word_from_json: Populates a Word template (.docx) using data from a JSON object (dictionary). Requires 'json_data', 'template_path', and 'output_path'. IMPORTANT: The keys in the 'json_data' dictionary for this tool should be the names INSIDE the template braces (e.g., 'PLACEHOLDER_CAMPAIGN_NAME').

You also have access to the following agents (interact via message history):

1. summarizer_agent: Summarizes previous campaign data. It uses the placeholder list identified earlier to focus its summary on relevant information.
2. brief_generator_agent: Generates a new campaign brief. It MUST use the placeholder list identified earlier as the required structure, generating content for each field based on the user prompt and summary.

Your workflow MUST be executed in the following precise steps:

Step 1: Identify Required Fields. Call the 'extract_placeholders_from_template' tool using './Data/CampaignBriefCreationTemplate.docx'. Remember the list of full placeholders (e.g., '{{PLACEHOLDER_NAME}}') returned.

Step 2: Gather Background Data. Call the 'aggregate_text_files' tool (no arguments).

Step 3: Summarize Background Data. Delegate the task to 'summarizer_agent'. **It will use the placeholder list found in the history to create a focused summary relevant to the required fields and return that summary.**

Step 4: Generate New Campaign Brief. Delegate the task to 'brief_generator_agent'. **It MUST use the placeholder list from the history as the target structure, generating content for each required field based on the user prompt and summary, and return the complete brief text. Make sure to generate content for all the elements in the list**

Step 5: Extract Generated Brief. Identify the complete text content of the campaign brief generated by 'brief_generator_agent' in the most recent AIMessage.

Step 6: Prepare Data for Word Template (Crucial Reasoning Step). Now, you MUST perform the following actions carefully:
    a. **Reference Placeholders:** Recall the list of full placeholders (e.g., '{{PLACEHOLDER_NAME}}') extracted in Step 1.
    b. **Parse Generated Brief:** Take the complete campaign brief text extracted in Step 5.
    c. **Construct JSON:** Analyze the brief text and CONSTRUCT a JSON object (Python dictionary). The keys MUST EXACTLY match the **content INSIDE** the placeholders from Step 6a (e.g., for '{{PLACEHOLDER_NAME}}', the key is 'PLACEHOLDER_NAME'). The values should be the corresponding text extracted or synthesized from the generated brief for each section. Ensure all placeholder *contents* from Step 6a are present as keys.
    d. **Validate JSON:** Ensure a valid Python dictionary.

Step 7: Save to Word Document. Call the 'populate_word_from_json' tool. Provide arguments EXACTLY:
    - `json_data`: The JSON object from Step 6c.
    - `template_path`: './Data/CampaignBriefCreationTemplate.docx'
    - `output_path`: './output/Final_Campaign_Brief.docx'

Step 8: Final Confirmation. Output the confirmation message returned by the 'populate_word_from_json' tool.

Begin the process following Step 1.
""")

# Recreate the supervisor workflow with the updated prompt
supervisor_workflow = create_supervisor(
    [summarizer_agent, brief_generator_agent], # List of agents supervised
    model = llm, # LLM for the supervisor agent
    tools = tools_for_supervisor, # Pass the tool to the supervisor
    prompt = supervisor_prompt
)

# Recompile and Rerun the app (Cell 5 code)
app = supervisor_workflow.compile()

print("---")
print("Please enter the requirements for the new campaign brief.")
print("Example: Create a new campaign brief for a winter holiday season promotion targeting families...")
print("Paste your brief details below and press Enter when done:")
print("---")
new_brief_prompt = input() # Read input from the console

# Optional: Add a check for empty input
if not new_brief_prompt.strip():
    print("No input received. Exiting.")
    exit() # Or provide a default prompt, or ask again

# --- MODIFICATION END ---


# Use the user-provided input in the invoke call
print("\n--- Invoking Workflow with User Prompt ---")
result = app.invoke({
        "messages": [
            {
                "role": "user",
                "content": f"User's New Campaign Brief Prompt: {new_brief_prompt}" # Use the variable holding user input
            }
        ]
    },{"recursion_limit": 100}) # Set an appropriate recursion limit

# Print the final results
print("\n--- Workflow Final Output ---")
for m in result['messages']:
    m.pretty_print()

In [ ]:
print("Hello World")